# Step 1: Load the Data

In [36]:
import pandas as pd

train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

# Step 2: Explore the Data

In [37]:
print(train.shape, test.shape)
print('-'*50)
print(train.head())
print('-'*50)
print(train.describe())
print('-'*50)
print(train.dtypes)
print('-'*50)
print('Checking for missing values:')
print(train.isnull().sum())
print('-'*50)
print('Checking for duplicates:')
print(train.duplicated().sum())

(8693, 14) (4277, 13)
--------------------------------------------------
  PassengerId HomePlanet CryoSleep  Cabin  Destination   Age    VIP  \
0     0001_01     Europa     False  B/0/P  TRAPPIST-1e  39.0  False   
1     0002_01      Earth     False  F/0/S  TRAPPIST-1e  24.0  False   
2     0003_01     Europa     False  A/0/S  TRAPPIST-1e  58.0   True   
3     0003_02     Europa     False  A/0/S  TRAPPIST-1e  33.0  False   
4     0004_01      Earth     False  F/1/S  TRAPPIST-1e  16.0  False   

   RoomService  FoodCourt  ShoppingMall     Spa  VRDeck               Name  \
0          0.0        0.0           0.0     0.0     0.0    Maham Ofracculy   
1        109.0        9.0          25.0   549.0    44.0       Juanna Vines   
2         43.0     3576.0           0.0  6715.0    49.0      Altark Susent   
3          0.0     1283.0         371.0  3329.0   193.0       Solam Susent   
4        303.0       70.0         151.0   565.0     2.0  Willy Santantines   

   Transported  
0        False

# Step 3: Data Preprocessing
## Separate features from target

In [38]:
X_train = train.drop('Transported', axis=1)
y_train = train['Transported']

## Combine train and test datasets to preprocess together

In [39]:
combined = pd.concat([X_train, test], keys=['train', 'test'])

## Drop likely useless columns
I also tried removing the Name column, but it reduced the accuracy, I believe that same surnames have high chances of the sharing the same target result since they are likely family and had high chances of being together at the moment of the accident.

In [40]:
# TODO CHECK
# combined.drop(columns=['HomePlanet', 'Destination', 'Age', 'VIP'], inplace=True)
# print(combined.head())

## Split columns with multiple information
Then drop the original columns as they are no longer needed.

In [41]:
combined[['Group', 'NumberInGroup']] = combined['PassengerId'].str.split('_', expand=True)
combined[['Deck', 'Num', 'Side']] = combined['Cabin'].str.split('/', expand=True)
combined.drop(columns=['PassengerId'], inplace=True)
combined.drop(columns=['Cabin'], inplace=True)

## Generate new column "TotalSpending"

In [42]:
# Create a new feature for total spending
combined['TotalSpending'] = (
        combined['RoomService'] +
        combined['FoodCourt'] +
        combined['ShoppingMall'] +
        combined['Spa'] +
        combined['VRDeck']
)

# # TODO Optional: Log transformation if the distribution is highly skewed
# combined['TotalSpending_log'] = np.log1p(combined['TotalSpending'])

#  TODO consider deleting old spend columns

## Handling missing values

In [43]:
# Fill missing values for categorical columns with a new class "Unknown"
combined['HomePlanet'] = combined['HomePlanet'].fillna('Unknown')
combined['CryoSleep'] = combined['CryoSleep'].fillna('Unknown')
combined['Deck'] = combined['Deck'].fillna('Unknown')
combined['Num'] = combined['Num'].fillna('Unknown')
combined['Side'] = combined['Side'].fillna('Unknown')
combined['Destination'] = combined['Destination'].fillna('Unknown')
combined['VIP'] = combined['VIP'].fillna('Unknown')
combined['Name'] = combined['Name'].fillna('Unknown')

# Flag Imputed Values
combined['Age_imputed'] = combined['Age'].isnull().astype(int)
combined['RoomService_imputed'] = combined['RoomService'].isnull().astype(int)
combined['FoodCourt_imputed'] = combined['FoodCourt'].isnull().astype(int)
combined['ShoppingMall_imputed'] = combined['ShoppingMall'].isnull().astype(int)
combined['Spa_imputed'] = combined['Spa'].isnull().astype(int)
combined['VRDeck_imputed'] = combined['VRDeck'].isnull().astype(int)
combined['TotalSpending_imputed'] = combined['TotalSpending'].isnull().astype(int)

# Fill missing values for numerical columns with the median
combined['Age'] = combined['Age'].fillna(combined['Age'].median())
combined['RoomService'] = combined['RoomService'].fillna(combined['RoomService'].median())
combined['FoodCourt'] = combined['FoodCourt'].fillna(combined['FoodCourt'].median())
combined['ShoppingMall'] = combined['ShoppingMall'].fillna(combined['ShoppingMall'].median())
combined['Spa'] = combined['Spa'].fillna(combined['Spa'].median())
combined['VRDeck'] = combined['VRDeck'].fillna(combined['VRDeck'].median())
combined['TotalSpending'] = combined['TotalSpending'].fillna(combined['TotalSpending'].median())

print(combined.isnull().sum())

HomePlanet               0
CryoSleep                0
Destination              0
Age                      0
VIP                      0
RoomService              0
FoodCourt                0
ShoppingMall             0
Spa                      0
VRDeck                   0
Name                     0
Group                    0
NumberInGroup            0
Deck                     0
Num                      0
Side                     0
TotalSpending            0
Age_imputed              0
RoomService_imputed      0
FoodCourt_imputed        0
ShoppingMall_imputed     0
Spa_imputed              0
VRDeck_imputed           0
TotalSpending_imputed    0
dtype: int64


## Handle Categorical Columns

In [44]:
# TODO handle this better

print(combined.head())

combined = pd.get_dummies(combined)

# print(combined.head())

        HomePlanet CryoSleep  Destination   Age    VIP  RoomService  \
train 0     Europa     False  TRAPPIST-1e  39.0  False          0.0   
      1      Earth     False  TRAPPIST-1e  24.0  False        109.0   
      2     Europa     False  TRAPPIST-1e  58.0   True         43.0   
      3     Europa     False  TRAPPIST-1e  33.0  False          0.0   
      4      Earth     False  TRAPPIST-1e  16.0  False        303.0   

         FoodCourt  ShoppingMall     Spa  VRDeck  ... Num Side TotalSpending  \
train 0        0.0           0.0     0.0     0.0  ...   0    P           0.0   
      1        9.0          25.0   549.0    44.0  ...   0    S         736.0   
      2     3576.0           0.0  6715.0    49.0  ...   0    S       10383.0   
      3     1283.0         371.0  3329.0   193.0  ...   0    S        5176.0   
      4       70.0         151.0   565.0     2.0  ...   1    S        1091.0   

        Age_imputed RoomService_imputed FoodCourt_imputed  \
train 0           0            

## Split combined dataset back into train and test sets

In [45]:
X_train = combined.xs('train')
X_test = combined.xs('test')

# Step 4: Train a Model

In [46]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# Split the data into training and validation sets
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Train model TODO try a random classification forest
model = LogisticRegression(max_iter=1000)
model.fit(X_train_split, y_train_split)

C:\Users\Alvar\OneDrive\Escritorio\kaggle-spaceship-titanic\venv\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=1000)

# Step 5: Evaluate the Model

In [47]:
from sklearn.metrics import accuracy_score

# Predict on the training set
y_train_pred = model.predict(X_train_split)

# Predict on the validation set
y_val_pred = model.predict(X_val_split)

# Evaluate the model
train_accuracy = accuracy_score(y_train_split, y_train_pred)
val_accuracy = accuracy_score(y_val_split, y_val_pred)

print(f'Training Accuracy: {train_accuracy:.4f}')
print(f'Validation Accuracy: {val_accuracy:.4f}')

# TODO try random classification forest results

Training Accuracy: 0.8132
Validation Accuracy: 0.7769


Best validation accuracy so far: 0.7867

# Step 6: Prepare Submission File

In [48]:
test_predictions = model.predict(X_test)
submission = pd.DataFrame({'PassengerId': test['PassengerId'], 'Transported': test_predictions})
submission.to_csv('submission.csv', index=False)